# LFQA 

In [ ]:
# https://towardsdatascience.com/ask-wikipedia-eli5-like-questions-using-long-form-question-answering-on-haystack-32cf1ca6c00e

## Setup

In [1]:
!pip install farm-haystack

## Example

In [2]:
from haystack.document_store.faiss import FAISSDocumentStore

document_store = FAISSDocumentStore(vector_dim=128, faiss_index_factory_str="Flat")

09/27/2021 23:49:42 - INFO - faiss.loader -   Loading faiss with AVX2 support.
09/27/2021 23:49:42 - INFO - faiss.loader -   Could not load library with AVX2 support due to:
ModuleNotFoundError("No module named 'faiss.swigfaiss_avx2'",)
09/27/2021 23:49:42 - INFO - faiss.loader -   Loading faiss.
09/27/2021 23:49:42 - INFO - faiss.loader -   Successfully loaded faiss.
09/27/2021 23:49:44 - INFO - farm.modeling.prediction_head -   Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex .


In [4]:
from datasets import load_dataset

wiki_snippets = load_dataset('wiki_snippets', name='wiki40b_en_100_0', streaming=True)['train']


Downloading:   0%|          | 0.00/2.11k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/580 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.03k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/639 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.38k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/9.42G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/527M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/522M [00:00<?, ?B/s]

Dataset wiki40b downloaded and prepared to /home/ec2-user/.cache/huggingface/datasets/wiki40b/en/1.1.0/3d99a1f96eba780a027018e9a18c1b9a6f5809625e78804a915495f6193dfa5e. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [7]:
from haystack.retriever.dense import EmbeddingRetriever

retriever = EmbeddingRetriever(document_store=document_store,
                               embedding_model="yjernite/retribert-base-uncased",
                               model_format="retribert")

document_store.update_embeddings(retriever)

09/27/2021 23:52:19 - INFO - haystack.retriever.dense -   Init retriever using embeddings of model yjernite/retribert-base-uncased


NotImplementedError: 

In [ ]:
from haystack.utils import print_answers, print_documents
from haystack.pipeline import DocumentSearchPipeline

p_retrieval = DocumentSearchPipeline(retriever)
res = p_retrieval.run(
    query="Why are Bermudan sloop ships widely prized?",
    top_k_retriever=10
)
print_documents(res, max_text_len=256)

In [ ]:
from haystack.generator.transformers import Seq2SeqGenerator
from haystack.pipeline import GenerativeQAPipeline

generator = Seq2SeqGenerator(model_name_or_path="yjernite/bart_eli5")
pipe = GenerativeQAPipeline(generator, retriever)

In [8]:
pipe.run(query="Why are Bermudan sloop ships widely prized?", top_k_retriever=4)

NameError: name 'pipe' is not defined